# MadMiner example

In this tutorial we'll demonstrate how to use MadMiner to generate train and test samples for the ML methods introduced in ["Constraining Effective Field Theories With Machine Learning"](https://arxiv.org/abs/1805.00013) and ["A Guide to Constraining Effective Field Theories With Machine Learning"](https://arxiv.org/abs/1805.00020), both by Johann Brehmer, Gilles Louppe, Juan Pavez, and Kyle Cranmer.

Before you execute this notebook, make sure you have running installations of MadGraph, Pythia, and Delphes. Note that at least for now, the MG-Pythia interface and Delphes require custom patches (available upon request). In addition, MadMiner and [DelphesMiner](https://github.com/johannbrehmer/delphesminer) have to be in your PYTHONPATH.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
from matplotlib import pyplot as plt
% matplotlib inline

from madminer import GoldMine, Smithy
from madminer.smithy import constant_benchmark_theta, multiple_benchmark_thetas
from madminer.smithy import constant_morphing_theta, multiple_morphing_thetas, random_morphing_thetas
from madminer.tools.plots import plot_2d_morphing_basis
from delphesminer import DelphesMiner

## 1. Define parameter space

After creating a `GoldMine` instance, the first important step is the definition of the parameter space. Each model parameter is characterized by a name as well as the LHA block and ID.

If morphing is used, one also has to specify the maximal power with which the parameter contributes to the squared matrix element. For instance, a parameter that contributes only to one vertex, will typically have `morphing_max_power=2`, while a parameter that contributes to two vertices usually has `morphing_max_power=4`. Exceptions arise for instance when the interference effects between the SM and dimension-six operators are modelled, but the square of the dimension-six amplitude (subleading in 1/Lambda) is not taken into account, in which case `morphing_max_power=1`. Finally, the `parameter_range` argument defines the range of parameter values that are used for the automatic optimization of the morphing basis.

In [ ]:
miner = GoldMine()

miner.add_parameter(
    lha_block='ANOINPUTS',
    lha_id=3,
    parameter_name='fW',
    morphing_max_power=4,
    parameter_range=(-1.2e-05,1.2e-05)
)
miner.add_parameter(
    lha_block='ANOINPUTS',
    lha_id=7,
    parameter_name='fWW',
    morphing_max_power=4,
    parameter_range=(-1.2e-05,1.2e-05)
)

## 2. Define benchmark points (evaluation points for |M|^2)

The next step is the definition of all the points at which the weights (squared matrix elements) should be evaluated by MadGraph. We call these points "benchmarks".

### 2a. Set benchmarks by hand

One can define benchmarks by hand:

In [ ]:
miner.add_benchmark(
    {'fW':0., 'fWW':0.},
    'sm'
)

### 2b. Benchmarks for morphing

If morphing is used, the function `set_benchmarks_from_morphing` has to be called. With the option `keep_existing_benchmarks=True`, MadMiner will keep all the benchmark points defined beforehand and run a simple optimization algorithm to fix the remaining ones for the basis (which may be none). Otherwise, MadMiner will optimize the full basis and forget about all previously defined benchmark points.

In [ ]:
miner.set_benchmarks_from_morphing(
    keep_existing_benchmarks=True,
    n_bases=1,
    n_trials=1000,
    n_test_thetas=100
)

Let's have a look at the resulting morphing basis and the "morphing error", i.e. the sum of squared morphing weights as a function of the parameter space:

In [ ]:
fig = plot_2d_morphing_basis(
    miner.morpher,
    xlabel=r'$f_{W} / \Lambda^2$ [GeV${}^{-2}$]',
    ylabel=r'$f_{WW} / \Lambda^2$ [GeV${}^{-2}$]',
    xrange=(-1.64949627e-05, 1.64949627e-05),
    yrange=(-1.64949627e-05, 1.64949627e-05),
)

## 3. Save settings and run MadGraph

The parameter space, benchmark points, and morphing setup are saved in a HDF5 file:

In [ ]:
miner.save('data/madminer_example.h5')

In a next step, MadMiner starts MadGraph and Pythia to generate events and calculate the weights. You have to provide the process card, run card, param card (the entries corresponding to the parameters of interest will be automatically adapted), and an empty reweight card.

If MadGraph is supposed to run in a different Python environment or requires other setup steps, you can use the `initial_command` argument.

In [ ]:
miner.run(
    mg_directory='/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2',
    proc_card_file='./cards/proc_card.dat',
    param_card_template_file='./cards/param_card_template.dat',
    reweight_card_template_file='./cards/reweight_card_template.dat',
    run_card_file='./cards/run_card.dat',
    sample_benchmark='sm',
    initial_command='source activate python2'
)

## 4. Run detector simulation and extract observables

The detector simulation and calculation of observables is not part of MadMiner. The reason is that different users might have very different requirements here: while a phenomenologist might be content with the fast detector simulation from Delphes, an experimental analysis might require the full simulation through Geant4.

We provide the DelphesMiner package, which wraps around Delphes and allows for the fast extraction of observables into the HDF5 file.

Any user is free to replace the DelphesMiner step with a tool of their choice. 

In [ ]:
dm = DelphesMiner()

After creating the DelphesMiner object, one can add a number of HepMC event samples...

In [ ]:
dm.add_hepmc_sample('./MG_process/Events/run_01/tag_1_pythia8_events.hepmc.gz')

... and have DelphesMiner run Pythia:

In [ ]:
dm.run_delphes(delphes_directory='/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes',
               delphes_card='./cards/delphes_card.dat')

The next step is the definition of observables through a name and a python expression. For the latter, you can use the objects `j[i]`, `e[i]`, `mu[i]`, `a[i]`, `met`, where the indices `i` refer to a ordering by the transverse momentum. All of these objects are scikit-hep [LorentzVectors](http://scikit-hep.org/api/math.html#vector-classes), see the link for a documentation of their properties.

In [ ]:
dm.add_observable('n_jets', 'len(j)')
dm.add_observable('pt_j1', 'j[0].pt')
dm.add_observable('pt_j2', 'j[1].pt')
dm.add_observable('m_jj', '(j[0] + j[1]).m')
dm.add_observable('delta_eta_jj', 'abs(j[0].eta - j[1].eta)')
dm.add_observable('delta_phi_jj', 'abs(j[0].phi() - j[1].phi())')

The function `analyse_delphes_samples` extracts all these observables from the Delphes ROOT file(s) generated before:

In [ ]:
dm.analyse_delphes_samples()

The values of the observables and the weights are then saved in the HDF5 file. It is possible to overwrite the same file, or to leave the original file intact and save all the data into a new file as follows:

In [ ]:
dm.save('data/madminer_example_with_data.h5', 'data/madminer_example.h5')

It's easy to check some distributions at this stage

In [ ]:
fig = plt.figure(figsize=(5,5))

for weights in dm.weights:
    plt.hist(dm.observations['pt_j1'], range=(0.,400.), bins=20, histtype='step', weights=weights)

plt.show()

## 5. Make training and test samples

The last important MadMiner class is the `Smithy`. From all the data we have in the HDF5 file now, it extracts unweighted samples including the augmented data ("gold") that is needed as training and evaluation data for the Machine Learning algorithms.

In [2]:
smithy = Smithy('data/madminer_example_with_data.h5', debug=True)

15:07  
15:07  ------------------------------------------------------------
15:07  |                                                          |
15:07  |  MadMiner                                                |
15:07  |                                                          |
15:07  |  Version from July 5, 2018                               |
15:07  |                                                          |
15:07  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
15:07  |                                                          |
15:07  ------------------------------------------------------------
15:07  
15:07  Hi! How are you today?
15:07  Loading data from data/madminer_example_with_data.h5
15:07  Found 2 parameters:
15:07     fW (LHA: ANOINPUTS 3, maximal power in squared ME: 4, range: (-1.2e-05, 1.2e-05))
15:07     fWW (LHA: ANOINPUTS 7, maximal power in squared ME: 4, range: (-1.2e-05, 1.2e-05))
15:07  Found 15 benchmarks:
15:07     sm: fW = 0.00e+00, fWW = 0.00e+0

The `Smithy` class defines four different high-level functions to generate train or test samples:
- `extract_samples_train_onlysamples()`, which only saves observations x, for instance for histograms or ABC;
- `extract_samples_train_local()` for methods like SALLY and SALLINO;
- `extract_samples_train_ratio()` for techniques like CARL, ROLR, CASCAL, and RASCAL; and
- `extract_samples_test()` for the evaluation of any method.

For the arguments `theta`, `theta0`, or `theta1`, you can use the helper functions `constant_benchmark_theta()`, `multiple_benchmark_thetas()`, `constant_morphing_theta()`, `multiple_morphing_thetas()`, and `random_morphing_thetas()`, all defined in the `smithy` module.

In [3]:
smithy.extract_samples_train_ratio(
    theta0=random_morphing_thetas(None, [('gaussian', 0., 1.e-5), ('gaussian', 0., 0.5e-5)]),
    theta1=constant_benchmark_theta('sm'),
    n_samples=10000,
    folder='./data/samples',
    filename='train'
)

15:07  Extracting training sample for ratio-based methods. Numerator hypothesis: ('random', (None, [('gaussian', 0.0, 1e-05), ('gaussian', 0.0, 5e-06)])), denominator hypothesis: ('benchmark', 'sm')
15:07  Starting sample extraction
15:07  New theta configuration
15:07    Sampling theta: morphing, [-0.00 0.00]
15:07    Auxiliary theta: benchmark, sm
15:07    # samples: 1


RuntimeError: Cannot calculate score without morphing or finite differences

In [ ]:
smithy.extract_samples_train_ratio(
    theta=constant_benchmark_theta('sm'),
    n_samples=10000,
    folder='./data/samples',
    filename='train'
)

In [ ]:
benchmark_index = 1

theta = smithy.benchmarks[list(smithy.benchmarks.keys())[benchmark_index]]
theta = [theta[key] for key in theta]
print(theta)

augmented_data_definitions = [
    ('ratio', 'benchmark', benchmark_index, 'benchmark', 0),
    ('ratio', 'benchmark', benchmark_index, 'morphing', np.array((0.,0.))),
    ('ratio', 'morphing', theta, 'benchmark', 0),
    ('ratio', 'morphing', theta, 'morphing', np.array((0.,0.))),
]

In [ ]:
theta, x, gold = smithy.extract_sample(theta_sampling_types=['benchmark'],
                                       theta_sampling_values=[0],
                                       n_samples_per_theta=10000,
                                       augmented_data_definitions=augmented_data_definitions)

In [ ]:
xmin =  [0, 0., 0., 0., 0., 0.]
xmax =  [8, 400., 250., 2000., 10., np.pi]
xbins = [9, 20, 20, 20, 20, 20]

fig = plt.figure(figsize=(12,12))

for i, (this_xmin, this_xmax, this_xbins) in enumerate(zip(xmin, xmax, xbins)):
    ax = plt.subplot(3, 3, i+1)
    
    cut = np.isfinite(x[0][:,i])
    this_data = x[0][cut,i]

    plt.hist(this_data,
             range=(this_xmin, this_xmax), bins=this_xbins,
             histtype='step', lw=1.5, ls='-')
    
    for j in range(gold[0].shape[1]):
        this_weights = gold[0][cut,j]
        plt.hist(this_data, weights=this_weights,
                 range=(this_xmin, this_xmax), bins=this_xbins,
                 histtype='step', lw=1.5, ls=['-', '--', '-.', ':'][j], alpha=0.4)

plt.tight_layout()
plt.show()